# 📚 LangChain ile Retrieval Augmented Generation'a Giriş 🦜🔗

Bu not defterinde LangChain kullanarak Retrieval Augmented Generation'ı nasıl kullanacağınızı öğreneceksiniz.

Kendi belgelerimiz hakkında sorular sormak için bir LLM kullanacağız!

## ⚙️ Kurulum

👉 Temel kütüphaneleri içe aktarmak için aşağıdaki hücreyi çalıştırın.

In [ ]:
import sys, os
from dotenv import load_dotenv

print("PYTHON:", sys.executable)
load_dotenv()
print("KEY?", bool(os.getenv("GOOGLE_API_KEY")))

In [ ]:
import langchain
import chromadb
import pypdf
from langchain_google_genai import GoogleGenerativeAIEmbeddings

print("imports OK ✅")

In [ ]:
%load_ext autoreload
%autoreload 2
import os
from pprint import pprint
from IPython.display import Markdown

👉 API anahtarımızı tekrar yüklemek için aşağıdaki hücreyi çalıştırın:

In [ ]:
from dotenv import load_dotenv

load_dotenv()  # Load environment variables from .env file

## 📚 Neden RAG?

Bir LLM kendi başına, öğrendiği her şey hakkında sorulara yanıt verebilir.

Bunun birkaç dezavantajı vardır:
- Eğitim verileri geçmişten gelir ve en son verilerle güncellenmez.
- Sadece eğitim aldığı verileri bilir.

Bir LLM'yi kendi verilerimizle çalışması için kullanmak istiyoruz. İşte bu noktada RAG (Retrieval-Augmented Generation) devreye girer.

1. **Retrieval-Augmented Generation (RAG)**, gerçek doğruluğu artırmak için bir dil modelini belge alıcı ile birleştirir.
2. **İlgili dış belgeleri alır** (örneğin, bilgi tabanından) yanıtlar üretmeden önce.
3. **Dil modeli hem istemi hem de alınan bağlamı kullanarak** daha bilgili ve temelli çıktılar üretir.

## 🇪🇺 Bağlam

Bu meydan okumada, Avrupa Parlamentosu'ndan belgelerle çalışacağız.

Bir gazeteci olduğunuzu ve Avrupa Parlamentosu'nun genel kurul oturumları sırasında belirli bir konu hakkında neler söylendiğini öğrenmek istediğinizi düşünün. Bu oturumlar yılda 12 kez Strasbourg'da gerçekleşir ve 4 gün sürer. Oturumların transkriptleri EP'nin web sitesinde mevcuttur.

Kesinlikle tüm bu transkriptleri karıştırmak istemezsiniz. O halde, hayatımızı kolaylaştırmak için RAG'ı kullanalım!

Bu, her zaman test etmek için yepyeni veriler alabileceğimiz için çalışmak üzere iyi verilerdir.

## 📘 Verileri alalım

1. [EP'nin web sitesine](https://www.europarl.europa.eu/plenary/en/debates-video.html) gidin. 
1. Bu sizi en son genel kurul oturumuna yönlendirecektir.
1. İlk tarihin altında, "▶️ Verbatim reports HTML" bölümünde `HTML`'e tıklayın.
1. Sayfanın sonuna kaydırın ve alttaki PDF dosyasını indirin.
1. Dosyayı `data` klasörüne kaydedin.

Bir belgeyle başlayacağız, ancak daha sonrası için diğer birkaç günün aynısını şimdiden indirebilirsiniz.

Belgeye bir göz atın. Kaç sayfası var? Belge hakkında bir fikir edinmek için hızlıca belgede gezinin.

## 🔢 Belgeleri gömme

Belgeleri gömmek, tüm belgeleri veya belge parçalarını vektörlere çevirmek anlamına gelir.

LangChain🦜🔗 yine çok yardımcı olacak.

Bir gömme aracı (embedder) başlatalım ve deneyelim. LLM olarak Gemini kullandığımız için, Google'ın metin gömme araçlarında kalalım.

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

👉 Basit bir metin parçasını gömmek için gömme aracının `.embed_query()` metodunu deneyin.

In [ ]:
# Embed a text like "What is the capital of France?" and save it to a variable `sample_embedding`

sample_text = "What is the capital of France?"
sample_embedding = embeddings.embed_query(sample_text)

👉 Bu `sample_embedding`'i keşfetmek için zaman ayırın. Nasıl görünüyor? Tipi nedir? Gömme boyutu nedir?

In [ ]:
type(sample_embedding), len(sample_embedding), sample_embedding[:10]

In [ ]:
import math

math.sqrt(sum(x * x for x in sample_embedding))

## 💾 PDF'den gerçek verilerimizi yükle

Artık bir gömmenin nasıl göründüğünü biliyoruz, gerçek verilerimizle çalışmanın zamanı geldi.

👉 [LangChain belgelerine](https://docs.langchain.com/oss/python/integrations/document_loaders/index#pdfs) gidin ve PyPDF kullanarak bir PDF'yi nasıl yükleyebileceğinizi öğrenin.

👉 Sonra devam edin ve daha önce indirdiğiniz PDF'lerden birini yükleyin.

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "data/CRE-10-2026-01-27_EN.pdf"
loader = PyPDFLoader(file_path
                    )
pages = loader.load()

👉 `pages`'i keşfedin:
- Veri tipi nedir?
- Kaç sayfanız var?
- Bir sayfanın tipi nedir?
- Bir sayfanın içeriğine nasıl erişebilirsiniz?
- Tam belgenin kaç karakteri var?
- Bir sayfanın `metadata`'sında neler var?

In [ ]:
# Veri tipi
type(pages)

In [ ]:
# Kaç sayfa var?
len(pages)

In [ ]:
# Bir sayfanın tipi
type(pages[0])

In [ ]:
# Bir sayfanın içeriği (ilk 500 karakter)
pages[0].page_content[:500]

In [ ]:
# Tüm belgenin toplam karakter sayısı
sum(len(page.page_content) for page in pages)

In [ ]:
# Bir sayfanın metadata'sı
pages[0].metadata

## ✂️ Verilerimizi böl

Tam belgemiz gömülmek için çok uzun. Metin gömme aracımız 2.048 tokena kadar giriş alabilir. Gemini modelleri için bu yaklaşık 8.196 karakterdir (token başına 4 karakter).

Bu yüzden belgemizi daha küçük parçalara bölmek istiyoruz.

Zaten çalışabileceğimiz bir dizi sayfamız var. Ama sayfa sonları biraz keyfi: genellikle cümlenin ortasında görünürler.

Ayrıca, sayfalar arasında örtüşme yoktur. Bu yüzden bir sayfanın ilk satırı önceki tüm bağlamı kaçırır. Tam metni biraz örtüşmeyle bölmek daha iyidir.

İlk olarak, PDF'yi tekrar yükleyeceğiz, bu sefer bölmeden.

In [ ]:
loader = PyPDFLoader(file_path, mode='single')
pdf = loader.load()
pdf_text = pdf[0].page_content
len(pdf_text)

Artık tüm PDF'imizi tek bir belge olarak aldığımıza göre, onu daha akıllı bir şekilde parçalara bölebiliriz.

👉 Yine, ["Özyinelemeli olarak bölme" konusundaki LangChain belgelerine](https://docs.langchain.com/oss/python/integrations/splitters/recursive_text_splitter) gidin ve `pdf` _belgelerimizi_ parçalara (LangChain'de `documents` olarak adlandırılır) nasıl böleceğinizi öğrenin.

2_000 karakter (bizim durumumuzda yaklaşık yarım sayfa) parçalara 400 örtüşmeyle bölün. İsterseniz diğer değerlerle deneyebilirsiniz.

`RecursiveCharacterTextSplitter`'ın `.split_documents()` metodunu kullanın: bu metod giriş olarak bir belge alır ve bölünmüş belgeler çıktısı verir.

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "data/CRE-10-2026-01-27_EN.pdf"

loader = PyPDFLoader(file_path, mode="single")
pdf = loader.load()

pdf_text = pdf[0].page_content
len(pdf_text)

👉 `all_splits`'i inceleyin:
- Veri tipi nedir?
- Kaç bölümünüz var?
- Bir bölümün tipi nedir?
- Bir bölümün içeriğine nasıl erişebilirsiniz?
- Şimdi toplamda kaç karakterimiz var?
- Bir bölümün `metadata`'sında neler var?

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

# Tek bir Document oluştur (metadata ekleyelim ki source kaybolmasın)
doc = Document(
    page_content=pdf_text,
    metadata={
        "source": file_path,
        "doc_name": "CRE-10-2026-01-27_EN",
    },
)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=400,
    separators=["\n\n", "\n", " ", ""],  # en doğal bölme sırası
)

all_splits = text_splitter.split_documents([doc])
len(all_splits)

In [ ]:
# Veri tipi nedir?
type(all_splits)

# Kaç bölüm var?
len(all_splits)

# Bir bölümün tipi nedir?
type(all_splits[0])

# Bir bölümün içeriğine nasıl erişilir? (ilk 300 karakter)
all_splits[0].page_content[:300]

# Şimdi toplamda kaç karakterimiz var?
sum(len(d.page_content) for d in all_splits)

# Bir bölümün metadata'sında neler var?
all_splits[0].metadata

## 🗄️ Her şeyi bir araya getir: belgelerimizi gömme ve vektör deposunda sakla

Elimizde şunlar var:
- Bir gömme aracı
- Veriyi yüklemek için bir yükleyici
- Belgemizi belgelere bölmek için bir metin bölücü

Neyi kaçırıyoruz?

Belgelerimizi gömebiliriz, ama onları bir yerde saklamak istiyoruz. İşte burada vektör deposu devreye girer: şunları saklamamıza olanak sağlar:
- belgeyi (parçayı),
- onun gömmesini,
- meta verilerini.

Sonraki adımda belgeleri verimli bir şekilde alabilecek olacağız.

👉 Bir `InMemoryVectorStore` nasıl oluşturabileceğinizi görmek için ["Vektör depoları" üzerine LangChain belgelerini](https://docs.langchain.com/oss/python/langchain/knowledge-base#3-vector-stores) kontrol edin.

In [ ]:
# Import the necessary libraries

from langchain_core.vectorstores import InMemoryVectorStore

# Create an in-memory vector store using the embedder `embeddings` we created earlier

vector_store = InMemoryVectorStore(embedding=embeddings)

# Add the `all_splits` to the vector store and store the result in a variable called `document_ids`

document_ids = vector_store.add_documents(all_splits)

In [ ]:
# Have a look at the first 3 document IDs

document_ids[:3]


In [ ]:
# Use the vector store's `get_by_ids` method. You have to give it a list of document IDs.

docs = vector_store.get_by_ids(document_ids[:1])
docs


👉 Bir vektör deposundaki belgenin içeriğine ve meta verilerine nasıl erişebilirsiniz?

In [ ]:
docs[0].page_content[:300], docs[0].metadata

## 🔎 Benzer belgeleri almak için vektör deposunu kullan

Artık belgeleri gömleğe çevirdiğimize göre, benzer belgeleri almak için vektör deposunu kullanabiliriz.

👉 Bunun nasıl çalıştığını görmek için ["Vektör depoları" üzerine LangChain belgelerini](https://docs.langchain.com/oss/python/langchain/knowledge-base#3-vector-stores) kontrol edin.

Bir sorgu kullanın, örneğin "Tarım politikası üzerine tartışmayı özetle.", ve en benzer belgeleri bulun. Ayrıca alınacak belge sayısını da belirtebilirsiniz.

In [ ]:
# Save your question into a variable called `query`

query = "What were the main topics discussed in this session? Summarize in bullet points."

# Use the vector store to find similar documents to the query. Store the result in a variable called `retrieved_docs`

k = 4  # istersen 3-8 arası oynat
retrieved_docs = vector_store.similarity_search(query, k=k)
retrieved_docs

[(d.metadata, d.page_content[:200]) for d in retrieved_docs]

Bu, RAG'ın sözde "Alma" (Retrieval) kısmını tamamlar: artık sorgumuza en benzer belgeleri bulabiliriz.

Çalışmanın çoğu artık tamamlandı!

## 💬 Sorumuza bir cevap üret

Şimdiye kadar benzer belgeleri almamızı sağlamak için sadece bir **gömme modeli** kullandık.

Şimdi, sorumuzla bir cevap almak için üretici bir LLM kullanacağız: ona aldığımız belgeler ve sorumuzla besleyeceğiz.

Bunu yapmanın en temel yolu tüm girdilerimizi birbirine bağlamak, sorumuzla eklemek ve sonucu görmek olacaktır.

Bir deneyelim.

👉 İlk olarak önceki meydan okumalarda olduğu gibi bir LLM başlatın.

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.2,
)

Sonra temel bir istem oluşturun:

In [ ]:
query = "What are the main topics discussed in this session?"

👉 Şimdi istemi kullanın:

In [ ]:
retrieved_docs = vector_store.similarity_search(query, k=4)

# İstersen hızlı kontrol:
len(retrieved_docs), type(retrieved_docs[0])

Bu fena değil, ama modele daha fazla rehberlik vererek daha kapsamlı bir istem yazarak daha iyisini yapabiliriz.

Bunu yapan ilk kişiler biz değilmişiz ve LangChain'in bizim için önceden hazırlanmış istem kütüphanesi var.

👉 Aşağıdaki hücreyi çalıştırın ve nasıl çalıştığını anlamaya çalışın. (LangSmithMissingAPIKeyWarning hakkında bir uyarı alacaksınız, bunu görmezden gelebilirsiniz.)

In [ ]:
CHUNK_CHARS = 1500  # artırırsan kalite artabilir ama token/maliyet de artar

context = "\n\n".join(
    doc.page_content[:CHUNK_CHARS] for doc in retrieved_docs
)

LangChain'in bizim için nasıl daha kesin bir istem oluşturduğunu görüyor musunuz? Bunu RAG'ımız için kullanalım!

In [ ]:
prompt = f"""
You are a question-answering assistant.
Answer using ONLY the CONTEXT below.

Return:
1) Answer (short)
2) Evidence: quote 1-2 short snippets from the context that support the answer.

If the answer is not in the context, say: "I don't know based on the provided context."

CONTEXT:
{context}

QUESTION:
{query}

FORMAT:
Answer: ...
Evidence:
- "..."
- "..."
""".strip()

In [ ]:
final_response = llm.invoke(prompt)
print(final_response.content)

👉 İlk olarak, tüm alınan belgeleri iki yeni satırla ayrılmış tek bir uzun dizgiye birleştirin.

👉 Sonra, sorgunuz ve alınan belgelerden başlayarak bir `prompt` oluşturun. Yukarıdaki örneğe bakmayı unutmayın.

👉 Son olarak az önce oluşturduğumuz `the_prompt` ile LLM modelini kullanın:

🎉 İlk RAG'ımızı tamamladık: LLM kendisine sağladığımız belgelerde ***temelli*** metin üretti.

## 💾 Gömmelerimizi kalıcı hale getir

Şimdiye kadar bellekte vektör deposuyla çalıştık. Bu yüzden not defterinizi kapattığınızda, tüm gömmeleri de kaybedeceksiniz.

⚠️ Bu gömmelerin sağlayıcınızın platformunda, bu durumda Google'ın makinelerinde çalışan modeller tarafından üretildiğini unutmayın. Ve bedava çalışmazlar. 💰

Bunun gibi bir, nispeten küçük belge için maliyet düşüktür, ama hızla artar. Şimdiye kadar sadece bir günün transkriptleriyle çalıştık. Oturum başına 3 tane daha, yılda 12 oturum, birden fazla yıl var...

Bunu çözmek için sadece vektör depomuzla kalıcı bir taneyi değiştireceğiz. Bu LangChain'in avantajıdır: bileşenleri değiştirmek çok kolay.

Bellekteki vektör depomuz deneme için harikaydı, şimdi başka bir taneyle değiştireceğiz. Çok popüler bir vektör deposu olan [Chroma](https://www.trychroma.com/)'yı kullanacağız. Bunu yerel olarak çalıştırabilir ve LangChain aracılığıyla kullanabiliriz.

Tüm akışımızı yeniden oluşturacağız. Her şeyi birkaç kod hücresinde tekrar bir araya getirmeye çalışmak iyi bir alıştırmadır. Aynı zamanda her şeyi yeniden kullanılabilir koda dönüştüreceğiz.

Sonunda iki fonksiyon istiyoruz:

1. `embed_and_store()`: Başka bir oturumun transkriptini vektör veritabanımıza ekle, böylece alacağımız daha fazla veri olsun.
2. `answer()`: Vektör depomuzla farklı sorularla sorgula.

#### 1. Bir Chroma vektör deposu başlat

👉 **Veri kalıcılığıyla** (yani verileri diskteki bir dizinde saklayarak) Chroma vektör deposunun nasıl oluşturulacağını görmek için [LangChain'in belgelerine](https://python.langchain.com/docs/integrations/vectorstores/chroma/) bakın.

In [ ]:
%pip -q install -U langchain langchain-core langchain-community langchain-google-genai chromadb pypdf

In [ ]:
import os
from datetime import date

from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [ ]:
# LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.2,
)

# Embedding (vektör oluşturmak için)
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/text-embedding-004"
)

In [ ]:
PERSIST_DIR = "chroma_store"          # klasör adı (proje içinde oluşur)
COLLECTION_NAME = "sessions"          # koleksiyon adı

vector_store = Chroma(
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIR,
    embedding_function=embeddings,
)

#### 2. `embed_and_store()` oluştur

👉 Bu fonksiyon için kodu tamamlayın:

In [ ]:
def embed_and_store(
    file_path: str,
    vector_store: Chroma,
    session_date: str | None = None,
    chunk_size: int = 1200,
    chunk_overlap: int = 150,
):
    """
    Load a PDF, split into chunks, add metadata, and store in Chroma.
    Returns: list of stored document ids (or empty list if none).
    """
    if session_date is None:
        session_date = str(date.today())

    # 1) Load PDF pages
    loader = PyPDFLoader(file_path)
    docs = loader.load()  # list[Document] (page-level)

    # 2) Split into chunks
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", " ", ""],
    )
    splits = splitter.split_documents(docs)

    # 3) Add metadata
    for d in splits:
        d.metadata["session_date"] = session_date
        d.metadata["source"] = d.metadata.get("source", file_path)

    # 4) Store
    ids = vector_store.add_documents(splits)

    # Chroma çoğu zaman otomatik persist eder ama garanti olsun:
    vector_store.persist()

    return ids

👉 Fonksiyonunuzu bir dosya veya hatta iki dosyayla deneyin:

#### 3. `answer()` oluştur

👉 Bu fonksiyon için kodu tamamlayın:

In [ ]:
PROMPT_TEMPLATE = """
You are a question-answering assistant.
Answer using ONLY the CONTEXT below.

Return:
1) Answer (short)
2) Evidence: quote 1-2 short snippets from the context that support the answer.

If the answer is not in the context, say: "I don't know based on the provided context."

CONTEXT:
{context}

QUESTION:
{query}

FORMAT:
Answer: ...
Evidence:
- "..."
- "..."
""".strip()

In [ ]:
def answer(
    query: str,
    vector_store: Chroma,
    llm: ChatGoogleGenerativeAI,
    k: int = 6,
    max_chars_per_doc: int = 1500,
) -> str:
    """Retrieve docs from vector store and answer using the LLM."""
    retrieved_docs = vector_store.similarity_search(query, k=k)

    context = "\n\n".join(d.page_content[:max_chars_per_doc] for d in retrieved_docs)

    prompt = PROMPT_TEMPLATE.format(context=context, query=query)

    response = llm.invoke(prompt)
    return response.content

👉 Fonksiyonunuzu beğendiğiniz bir sorguyla deneyin:

In [ ]:
ids = embed_and_store(
    file_path="data/CRE-10-2026-01-27_EN.pdf",
    vector_store=vector_store,
    session_date="2026-01-27",
)
len(ids), ids[:3]

In [ ]:
query = "What are the main topics discussed in this session?"
print(answer(query, vector_store, llm))

🏁 Tebrikler! Artık LangChain kullanarak RAG'da ustalaştınız ve vektör deponuza daha fazla belge eklemek ve onu sorgulamak için yeniden kullanılabilir fonksiyonlar yapmayı öğrendiniz.

## [İsteğe Bağlı] Meta veri ekleme

Kurduğumuz RAG, vektör deposundaki tüm belgeleri sorgular. Orada birden fazla yılın bilgisinin olduğunu düşünün. Yıllara veya tarihlere göre filtreyebilsek kullanışlı olurdu, değil mi?

Bunu nasıl yaparız? Vektör deposundaki belgelerin meta veri içerdiğini unutmayın. Eğer tarihi ekleyebilseydik, daha sonra filtrelemek için kullanabilirdik.

İpucu: Meta verilerinizi pipeline'ınızda olabildiğince erken ekleyin. Verileriniz vektör deposunda saklandıktan sonra eklemeye çalışmayın.

👉 `embed_and_store()` fonksiyonunuzu uyarlayın.

In [ ]:
def embed_and_store(
    file_path: str,
    vector_store: Chroma,
    session_date: str | None = None,
    chunk_size: int = 1200,
    chunk_overlap: int = 150,
):
    """
    Load a PDF, split into chunks, add metadata, store in Chroma.
    Returns: list of stored document ids.
    """
    if session_date is None:
        session_date = str(date.today())

    # 1) Load PDF (page-level docs)
    loader = PyPDFLoader(file_path)
    docs = loader.load()  # list[Document], metadata includes "source" and "page" usually

    # 2) Split into chunks
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", " ", ""],
    )
    splits = splitter.split_documents(docs)

    # 3) Add metadata EARLY (before storing)
    for i, d in enumerate(splits):
        d.metadata["session_date"] = session_date
        d.metadata["chunk_id"] = i

        # kaynağı garantiye al
        d.metadata["source"] = d.metadata.get("source", file_path)

        # page alanı çoğu zaman loader'dan gelir; yoksa -1 koy
        d.metadata["page"] = d.metadata.get("page", -1)

    # 4) Store
    ids = vector_store.add_documents(splits)

    # ❌ artık gerek yok (Chroma otomatik persist ediyor)
    # vector_store.persist()

    return ids

👉 Fonksiyonunuzu deneyin ve vektör deponuzun ek meta veri içerdiğini kontrol edin.

In [ ]:
def answer(
    query: str,
    vector_store: Chroma,
    llm,
    k: int = 6,
    session_date: str | None = None,
    max_chars_per_doc: int = 1500,
):
    """
    Query Chroma; optionally filter by session_date metadata.
    Returns model output text.
    """
    where = {"session_date": session_date} if session_date else None

    retrieved_docs = vector_store.similarity_search(
        query,
        k=k,
        filter=where,   # <- meta filtre burada
    )

    # context'i kontrollü büyüt (token şişmesini azaltır)
    context = "\n\n".join(d.page_content[:max_chars_per_doc] for d in retrieved_docs)

    prompt = f"""
You are a question-answering assistant.
Answer using ONLY the CONTEXT below.

Return:
1) Answer (short)
2) Evidence: quote 1–2 short snippets from the context that support the answer.

If the answer is not in the context, say: "I don't know based on the provided context."

CONTEXT:
{context}

QUESTION:
{query}

FORMAT:
Answer: ...
Evidence:
- "..."
- "..."
""".strip()

    resp = llm.invoke(prompt)
    return resp.content

Şimdi alıcıyı kullanıcının sorduğu tarihe göre sınırlamamız gerekiyor.

👉 `answer()` fonksiyonunuzu bir tarih alabilecek ve yeni meta verilere dayalı olarak belgeleri filtreleyebilecek şekilde uyarlayın.

In [ ]:
ids = embed_and_store(
    file_path="data/CRE-10-2026-01-27_EN.pdf",
    vector_store=vector_store,
    session_date="2026-01-27",
)

len(ids), ids[:3]

In [ ]:
query = "What are the main topics discussed in this session?"
print(answer(query, vector_store, llm))

In [ ]:
query = "What are the main topics discussed in this session?"
print(answer(query, vector_store, llm, session_date="2026-01-27"))

Harika! Güçlü bir RAG sistemi oluşturmak için benzerlik aramasını meta veri aramasıyla birleştirdiniz!